# 0. Install and Import Dependencies

In [6]:
# !pip install mediapipe opencv-python pandas scikit-learn
np.version.version

'1.21.5'

In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils # helpers for drawing
mp_pose = mp.solutions.pose

In [3]:
VIDEO_PATH = 'bicep-curl.mp4'
EXPORT_PATH = 'bicep-curl.csv'
MODEL_PATH = 'bicep-curl.pkl'

# X. Make Some Detections

In [5]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = pose.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
        
        cv2.imshow('LUCIO', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

# 1. Save Video

In [10]:
cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('bicep-curl.mp4', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()

    try:
        cv2.imshow('Bicep-Curl', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV

In [4]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [5]:
landmarks = ['class']
for val in range(1, 33 +1):
    landmarks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']

In [6]:
landmarks[:]

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [36]:
with open('coords.csv', mode= 'w', newline= '') as f:
    csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [37]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        #keypoints = np.insert(keypoints, 0, action)
        #print('normal mid execution')
        with open('coords.csv', mode= 'a', newline= '') as f:
            csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [38]:
cap = cv2.VideoCapture('bicep-curl.mp4')

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    while cap.isOpened():
        ret, image = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        ## results = pose.process(image)
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
                
        k = cv2.waitKey(1)
        if k == ord('a'):
            export_landmark(results, 'correct')
        elif k == ord('s'):
            export_landmark(results, 'error1')
        elif k ==ord('d'):
            export_landmark(results, 'error2')


        cv2.imshow('Bicep Training', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 

In [1]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('coords.csv')

In [9]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,correct,0.547642,0.299714,0.047692,0.979980,0.547861,0.284823,0.054196,0.981189,0.549311,...,-0.047761,0.357624,0.525241,1.248412,0.108711,0.154310,0.522787,1.270879,-0.130634,0.207501
1,correct,0.534964,0.303959,0.010218,0.983696,0.537969,0.290659,0.022342,0.984648,0.539812,...,-0.088756,0.371074,0.522265,1.244381,0.012003,0.144614,0.510753,1.266497,-0.197106,0.217082
2,correct,0.534811,0.304696,-0.049501,0.986687,0.535646,0.290421,-0.033648,0.987421,0.536971,...,-0.100995,0.382861,0.515379,1.219017,0.063173,0.135385,0.511891,1.241684,-0.197145,0.233029
3,correct,0.533849,0.304165,-0.215032,0.989032,0.533848,0.288992,-0.189126,0.989565,0.535056,...,-0.067717,0.386127,0.514833,1.178893,0.145372,0.126403,0.523163,1.217215,-0.161633,0.240876
4,correct,0.532852,0.303603,-0.271637,0.990115,0.533177,0.288819,-0.246326,0.990581,0.534699,...,-0.058816,0.380614,0.515317,1.178531,0.138695,0.125230,0.523094,1.208880,-0.152382,0.244116


In [10]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
135,error1,0.434769,0.267321,-0.195475,0.995539,0.428908,0.244158,-0.175536,0.992785,0.429965,...,-0.029157,0.299566,0.543061,1.260551,0.117023,0.094723,0.535086,1.260314,-0.155392,0.247013
136,error1,0.436619,0.266434,-0.141392,0.994960,0.430178,0.243533,-0.125171,0.991895,0.430939,...,-0.043909,0.295229,0.538023,1.253991,0.111384,0.090284,0.531905,1.257965,-0.165803,0.242494
137,error1,0.472547,0.260348,-0.261811,0.995175,0.474272,0.237484,-0.236935,0.992438,0.476133,...,-0.022359,0.305397,0.518853,1.200272,0.137339,0.098189,0.521626,1.227713,-0.125006,0.244793
138,error1,0.486235,0.260839,-0.258490,0.995731,0.489082,0.239797,-0.232921,0.993261,0.490977,...,-0.042771,0.301314,0.499082,1.203275,0.101451,0.097548,0.499415,1.224696,-0.167032,0.236978
139,error1,0.501435,0.265121,-0.312552,0.996707,0.505596,0.245545,-0.286693,0.994722,0.507978,...,-0.007673,0.303494,0.490730,1.220000,0.103536,0.102538,0.480808,1.242991,-0.130431,0.234208


In [11]:
df[df['class'] == 'correct']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,correct,0.547642,0.299714,0.047692,0.979980,0.547861,0.284823,0.054196,0.981189,0.549311,...,-0.047761,0.357624,0.525241,1.248412,0.108711,0.154310,0.522787,1.270879,-0.130634,0.207501
1,correct,0.534964,0.303959,0.010218,0.983696,0.537969,0.290659,0.022342,0.984648,0.539812,...,-0.088756,0.371074,0.522265,1.244381,0.012003,0.144614,0.510753,1.266497,-0.197106,0.217082
2,correct,0.534811,0.304696,-0.049501,0.986687,0.535646,0.290421,-0.033648,0.987421,0.536971,...,-0.100995,0.382861,0.515379,1.219017,0.063173,0.135385,0.511891,1.241684,-0.197145,0.233029
3,correct,0.533849,0.304165,-0.215032,0.989032,0.533848,0.288992,-0.189126,0.989565,0.535056,...,-0.067717,0.386127,0.514833,1.178893,0.145372,0.126403,0.523163,1.217215,-0.161633,0.240876
4,correct,0.532852,0.303603,-0.271637,0.990115,0.533177,0.288819,-0.246326,0.990581,0.534699,...,-0.058816,0.380614,0.515317,1.178531,0.138695,0.125230,0.523094,1.208880,-0.152382,0.244116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,correct,0.513132,0.254342,-0.240834,0.996782,0.513552,0.233811,-0.216436,0.995353,0.515465,...,-0.067086,0.246266,0.494824,1.188992,0.078504,0.073193,0.492202,1.224680,-0.181026,0.163300
58,correct,0.514829,0.253744,-0.220265,0.997831,0.515148,0.233288,-0.196898,0.996820,0.516790,...,-0.054967,0.255102,0.496085,1.206167,0.089513,0.079266,0.489285,1.238786,-0.166336,0.174590
59,correct,0.516401,0.257067,-0.181573,0.997946,0.515964,0.235817,-0.161294,0.996946,0.517150,...,-0.050873,0.248631,0.496464,1.204144,0.093213,0.077855,0.490063,1.233361,-0.163877,0.169926
60,correct,0.519992,0.256982,-0.160679,0.997823,0.519351,0.235369,-0.140445,0.996712,0.519988,...,-0.027375,0.246592,0.496291,1.204705,0.127994,0.079448,0.489078,1.233449,-0.143240,0.171279


In [12]:
X = df.drop('class', axis= 1) 
y = df['class']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 7)

In [14]:
y_test

125     error1
94      error1
129     error1
101     error1
22     correct
119     error1
28     correct
139     error1
11     correct
32     correct
45     correct
104     error1
77      error1
98      error1
132     error1
116     error1
36     correct
76      error1
51     correct
111     error1
107     error1
91      error1
128     error1
117     error1
59     correct
54     correct
49     correct
5      correct
37     correct
133     error1
17     correct
85      error1
79      error1
130     error1
124     error1
65      error1
40     correct
52     correct
131     error1
66      error1
62      error1
74      error1
Name: class, dtype: object

## 3.2 Train Machine Learning Classification Model

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [16]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [17]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [18]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [19]:
fit_models['rc'].predict(X_test)

array(['error1', 'error1', 'error1', 'error1', 'correct', 'error1',
       'correct', 'error1', 'correct', 'correct', 'correct', 'error1',
       'error1', 'error1', 'error1', 'error1', 'correct', 'error1',
       'correct', 'error1', 'error1', 'error1', 'error1', 'error1',
       'correct', 'correct', 'correct', 'correct', 'correct', 'error1',
       'correct', 'error1', 'error1', 'error1', 'error1', 'error1',
       'correct', 'correct', 'error1', 'error1', 'error1', 'error1'],
      dtype='<U7')

## 3.3 Evaluate and Serialize Model

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [21]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,
         accuracy_score(y_test.values, yhat),
         precision_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct")
        )

lr 0.9761904761904762 0.9761904761904762 0.9761904761904762 0.9761904761904762
rc 1.0 1.0 1.0 1.0
rf 1.0 1.0 1.0 1.0
gb 0.9761904761904762 0.9761904761904762 0.9761904761904762 0.9761904761904762


c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.war

In [22]:
yhat = fit_models['rf'].predict(X_test)

In [23]:
yhat[ :10]

array(['error1', 'error1', 'error1', 'error1', 'correct', 'error1',
       'correct', 'error1', 'correct', 'correct'], dtype=object)

In [24]:
with open('bicep-curl.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [27]:
with open('bicep-curl.pkl', 'rb') as f:
    pickle.load(f)

In [ ]:
# X = pd.DataFrame([row], columns= landmarks[1:])

In [28]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
state = ''

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    while cap.isOpened():
        ret, image = cap.read()
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = pose.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
                
        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row], columns= landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)

            correct = False; error1 = False; error2 = False

            if body_language_class == 'correct' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = True
                error1 = False
                error2 = False
            elif body_language_class == 'error1' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = True
                error2 = False
            elif body_language_class == 'error1' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = False
                error2 = True
            
            if (error1):
                state = 'Elbow is out of place'
            elif (correct):
                state = 'Correct'
            elif(error2):
                state = 'Leaning'

            # elif current_stage = 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= .7:
            #     current_stage = 'up'
            #     counter += 1
            
            counter = 0
            
            
                # print(current_stage)

            # Get status box
            #cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)
            
            # Display Class
            cv2.putText(image, 
                        'CLASS',
                        (0,0),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            
            # cv2.putText(image, 
            #             body_language_class(' ')[0],
            #             (90,40),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,
            #             (255,255,255),
            #             2,
            #             cv2.LINE_AA
            #            )
            #Display Propability
            cv2.putText(image, 
                        'PROB',
                        (15,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(round(body_language_prob[np.argmax(body_language_prob)], 2)),
                        (10,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255,255,255),
                        2,
                        cv2.LINE_AA
                       )
            
            #Display Count
            cv2.putText(image, 
                        'COUNT',
                        (180,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(state),
                        (175,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0,0,255),
                        2,
                        cv2.LINE_AA
                       )
            
            
        except Exception as e:
            pass


        cv2.imshow('Lucio', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

In [33]:
row.shape

(33, 4)

In [38]:
len(landmarks[1:])

132

In [41]:
len([row])

1

In [47]:
row.flatten().shape

(132,)

In [52]:
row.shape

(33, 4)